In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
# import plotly.express as px

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT *
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)

SDB.close()
cols = df.columns
for c in cols:
    print(c)

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,`šaldytuvų tipai`, tipas, `gaminio tipas`, `garsumo klasė`, `šaldiklio tipas`, 
`durų tipas`, `durų kiekis`, `šaldymo technologijos tipas`, `šaldytuvo tipas`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

In [ ]:
lst = df['tipas'].tolist()
print(set(lst))

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

# sql="""SELECT kaina, `montavimo tipas`, `durų tipas`, `durų montavimas`, `šaldytuvų tipai`
# FROM "SaldytuvaiVarle";
# """

sql="""SELECT kaina, `montavimo tipas`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)

SDB.close()


In [ ]:
df.columns

In [ ]:
df.dropna(subset='montavimo tipas', inplace=True)
df.head()

In [ ]:
tipai = set(df['montavimo tipas'].tolist())
print(tipai)

In [ ]:
def set_saldytuvo_tipas(x):
    if 'stat' in x:
        return 'Laivai pastatomi'
    elif 'montuo' in x:
        return 'Įmontuojami'
    elif 'ntegruo' in x:
        return 'Integruojami'
    else:
        return 'Kita'
    
df['mtipas'] =df['montavimo tipas'].apply(set_saldytuvo_tipas)
df['kaina'] = df['kaina'].apply(lambda x: float(x))
df.head()

In [ ]:

df1 = df[['kaina', 'mtipas']].groupby('mtipas').mean(numeric_only=True).reset_index()
# df1_gr = df1.groupby('mtipas').mean(numeric_only=True).reset_index()
df1

In [ ]:
c =df['mtipas'].value_counts()
fig, (axis1, axis2) = plt.subplots(1,2, figsize=(10.5, 4.5))
axis1.pie(c.values, labels=c.index, autopct='%.2f%%')
sns.boxplot(data=df, x='mtipas', y='kaina', showmeans=True, showfliers=False, ax=axis2)
axis1.set_title('Šaldytuvų pasiskirstymas pagal montavimo tipą (Varle.lt)')
axis2.set_title('Kainų pasiskirstymas pagal šaldytuvų  montavimo tipą (Varle.lt)')
axis2.set_xlabel('Tipas')
axis2.set_ylabel('Kaina')
fig.tight_layout()
plt.show()

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT `Šaldytuvo tipas:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
lst = df['Šaldytuvo tipas:'].tolist()
print(set(lst))

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, tipas
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

In [ ]:
df.dropna(subset='tipas', inplace=True)

def set_saldytuvo_tipas(x):
    # if 'labas' in x:
    #     return 'nar'
    if 'apa' in x:
        return 'Šaldytuvas su šaldikliu apačioje'
    elif 'virš' in x or 'with top' in x:
        return 'Šaldytuvas su šaldikliu viršuje'
    elif 'Mini' in x:
        return 'Mini'
    elif 'without' in x or 'Be k' in x:
        return 'Šaldytuvas be šaldiklio'
    elif 'kamera' in x or 'inside' in x:
        return 'Šaldytuvas su šaldikliu viduje'
    elif 'Dvid'in x or 'Double' in x:
        return 'Dviduriai šaldytuvai'
    elif 'stat' in x:
        return 'Laisvai pastatomas'
    else:
        return x
    
df['stipas'] =df['tipas'].apply(set_saldytuvo_tipas)

In [ ]:
top = df['stipas'].value_counts().head(10).index.tolist()
df['stipas'] = df['stipas'].apply(lambda x: x if x in top else 'Kita')
c = df['stipas'].value_counts()

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.pie(c.values, labels=c.index, autopct='%.1f%%')
plt.title('Šaldytuvų pasiskirstymas pagal tipą (Varle.lt)')
plt.show()

In [ ]:
df['kaina'] = df['kaina'].apply(lambda x: float(x))
# kaina_gr =df[['kaina', 'stipas']].groupby('stipas').mean(numeric_only=True).round()

# sns.boxplot(data=kaina_gr, y='stipas', x='kaina', orient='h')
sns.boxplot(data=df, y='stipas', x='kaina', showmeans=True, showfliers=False)
plt.xlabel('Kaina')
plt.ylabel('')
plt.title('Šaldytuvų kainos pagal tipą (Varle.lt)')
plt.show()


In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, `talpa (l)`, `naudinga šaldytuvo talpa`, `šaldytuvo talpa [l]`,
`bendra talpa neto`, `šaldytuvo talpa neto`, `šaldytuvo talpa`, `bendros grynosios talpos`, `bendra talpa`, `šaldymo talpa`,
`šaldytuvo neto talpa`, `bendra neto talpa`, `bendroji talpa neto`, `šaldytuvo talpa (neto)`,
`talpa (neto)`, `šaldytuvo talpa (l)`, `grynoji talpa`, `bendroji talpa neto (l)`, `šaldytuvo talpa neto (l)`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

In [ ]:
for col in df.columns:
    if col != 'kaina':
        df['V'] = df['talpa (l)'].fillna(df[col])
    
df['kaina'] = df['kaina'].apply(lambda x: float(x))

In [ ]:
df.dropna(subset='V', inplace=True)
dfv = df[['kaina', 'V']]
dfv['V'] = df['V'].apply(lambda x: float(x))
dfv['Vgr'] = dfv['V'].apply(lambda x: int(np.ceil(x/10) * 10))

dfv_gr = dfv.groupby('Vgr').mean(numeric_only=True).round().reset_index()
dfv_gr.head()
sns.regplot(data=dfv_gr[dfv_gr['Vgr'] > 10], x='Vgr', y='kaina', order=2)
plt.title('Šaldytuvo kainos priklausomybė nuo talpos (Varle.lt)')
plt.ylabel('Kaina')
plt.xlabel('Šaldytuvo talpa (l)')
plt.show()

In [ ]:
# energijos klase

SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, 
`energijos klasė`, 
`energijos vartojimo efektyvumo klasė (reglamentas (es) 2017/1369)`,
`energijos efektyvumo klasė`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
df.head()

In [ ]:
for col in df.columns:
    if col != 'kaina':
        df['energijos klasė'] = df['energijos klasė'].fillna(df[col])
    
df['energijos klasė'] = df['energijos klasė'].apply(lambda x: x.upper() if x is not None else x)
df['kaina'] = df['kaina'].apply(lambda x: float(x))

dfe = df[['kaina', 'energijos klasė']]
# dfe_gr = dfe.groupby('energijos klasė').mean(numeric_only=True).round().reset_index()
fig, ax = plt.subplots()
sns.boxplot(data=dfe, x='energijos klasė', y='kaina', showmeans=True, showfliers=False)
# for container in ax.containers:
#     ax.bar_label(container)
plt.title('Vidutinė šaldytuvo kaina pagal energijos klasę (Varle.lt)')
plt.show()

In [ ]:
# Energijos sanaudos

SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, 
`vidutinės metinės energijos sąnaudos kilovatvalandėmis per metus (kwh/m.)`,
`metinės energijos sąnaudos`,
`metinis energijos suvartojimas (kwh)`,
`energijos suvartojimas per metus`,
`energijos sąnaudos`,
`metinis energijos suvartojimas`,
`suvartojama energija`,
`bendras suvartojamos energijos kiekis per metus`,
`energijos sąnaudos (kwh/metus)`,
`suvartojama energija (kwh)`,
`bendras suvartojamos energijos kiekis per metus (kwh)`,
`energijos sąnaudos per metus`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
df.head()

In [ ]:
for col in df.columns:
    if col != 'kaina':
        df['energija'] = df['metinės energijos sąnaudos'].fillna(df[col])

df['kaina'] = df['kaina'].apply(lambda x: float(x))
dfe = df[['kaina', 'energija']]
dfe.dropna(subset='energija', inplace=True)
dfe.head()

In [ ]:
dfe['energija'] = dfe['energija'].apply(lambda x: x.replace('kWh/metus', '').replace('kWh', '').replace(',', '.'))
dfe['energija'] = dfe['energija'].apply(lambda x: float(x))

In [ ]:

dfe['E'] = dfe['energija'].apply(lambda x: int(np.ceil(x/5) * 5))

dfe_gr = dfe.groupby('E').mean(numeric_only=True).round().reset_index()

sns.regplot(data=dfe_gr[dfe_gr['E'] < 500], x='E', y='kaina', order=3)
plt.title('Šaldytuvo kainos priklausomybė nuo energijos suvartojimo (Varle.lt)')
plt.ylabel('Kaina')
plt.xlabel('Energijos suvartojimas kWh/metus')
plt.show()

In [ ]:
# garsas/triuksmas
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, 
`ore skleidžiamas akustinis triukšmas`,
`triukšmo lygis [db]`,
`maksimalus triukšmo lygis`,
`triukšmo lygis (db)`,
`triukšmo lygis`,
`triukšmo galia`,
`triukšmo lygis, db`,
`triukšmingumo lygis`,
`triukšmo lygis (db (a) re 1 pw)`,
`garso lygis`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
df.head()


In [ ]:
df['kaina'] = df['kaina'].apply(lambda x: float(x))
for col in df.columns:
    if col != 'kaina':
        df['garsas'] = df['ore skleidžiamas akustinis triukšmas'].fillna(df[col])

df.dropna(subset='garsas', inplace=True)
df.head()

In [ ]:
df['garsas'] = df['garsas'].apply(lambda x: int(x[:2]))

In [ ]:
dfg = df[['kaina', 'garsas']]
dfg['g2'] = dfg['garsas'].apply(lambda x: int(np.ceil(x/2) * 2))

dfg_gr = dfg.groupby('g2').mean(numeric_only=True).round().reset_index()
fig, ax = plt.subplots()
sns.regplot(data=dfg_gr, x='g2', y='kaina', order=1)
ax.set_xlabel('Triukšmo lygis (db)')
ax.set_ylabel('Kaina')
plt.title('Šaldytuvų kainų priklausomybė nuo triukšmo lygio')
plt.show()

In [ ]:
# valdymo pobūdis
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, 
`valdymas`,
`valdymo tipas`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
df.head()

In [ ]:
df['kaina'] = df['kaina'].apply(lambda x: float(x))
df['valdymas'] = df['valdymas'].fillna(df['valdymo tipas'])
df.dropna(subset='valdymas', inplace=True)

In [ ]:
def set_valdymas(x):
    if 'Elek' in x or 'Elec' in x or 'elek' in x:
        return 'Elektroninis'
    elif 'Palie' in x or 'Touch' in x or 'Juti' in x or 'juti' in x:
        return 'Sensorinis'
    elif 'Mech' in x or 'Meh' in x or 'Myg' in x or 'Rot' in x:
        return 'Mechaninis'
    elif 'Exter' in x:
        return 'External touch control LCD'
    elif 'varto' in x or 'rank' in x:
        return 'Kita'
    else:
        return x

df['newVald'] = df['valdymas'].apply(set_valdymas)


In [ ]:
dfv = df[['kaina', 'newVald']]
# dfv_gr = dfv.groupby('newVald').mean(numeric_only=True).round().reset_index()

fig, ax = plt.subplots()
sns.boxplot(data=dfv, y='newVald', x='kaina', showmeans=True, showfliers=False)
# sns.barplot(data=dfv_gr, y='newVald', x='kaina', orient='h')
# for container in ax.containers:
#     ax.bar_label(container)
plt.ylabel('')
plt.xlabel('Kaina')
plt.title('Vidutinė šaldytuvo kaina pagal valdymo tipą (Varle.lt)')
plt.show()

In [ ]:
# Gamintojai
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina, 
gamintojas,
spalva,
`gaminio spalva`,
`produkto spalva`,
`korpuso spalva`
FROM "SaldytuvaiVarle";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()
df.head()

In [ ]:
for col in df.columns:
    if col not in ['kaina', 'gamintojas']:
        df['spalva'] = df['spalva'].fillna(df[col])

df['kaina'] = df['kaina'].apply(lambda x: float(x))

In [ ]:
def set_spalva(x):
    if x is not None:
        if 'Balt' in x or 'balt' in x:
            return 'Balta'
        elif 'Pilk' in x or 'pilk' in x:
            return 'Pilka'
        elif 'Juod' in x or 'juod' in x:
            return 'Juoda'
        else:
            return x

df['spalva'] = df['spalva'].apply(set_spalva)

c= df['spalva'].value_counts()
c

In [ ]:
brand_counts = df['gamintojas'].value_counts()
brands_to_plot = brand_counts[brand_counts > 20].index

df_gamintojas = df[df['gamintojas'].isin(brands_to_plot)][['kaina', 'gamintojas']]
# df_gamintojas_gr = df_gamintojas.groupby('gamintojas').mean().round()
fig, ax = plt.subplots()
sns.boxplot(data=df_gamintojas, y='kaina', x='gamintojas', showfliers=False, showmeans=True)
plt.title('Šaldytuvų kainos pasiskirtymas pagal gamintoją (Varle.lt)')
plt.tick_params(axis='x', rotation=90)
plt.ylabel('')
plt.xlabel('Kaina')
plt.show()


In [ ]:
colors_counts = df['spalva'].value_counts()
colors_to_plot = colors_counts[colors_counts > 40].index
brand_counts = df['gamintojas'].value_counts()
brands_to_plot = brand_counts[brand_counts > 40].index

df_gamintojas = df[(df['gamintojas'].isin(brands_to_plot)) & df['spalva'].isin(colors_to_plot)][['kaina', 'gamintojas', 'spalva']]
df_gamintojas_gr = df_gamintojas.groupby(['gamintojas', 'spalva']).mean().round()
fig, ax = plt.subplots()
sns.barplot(data=df_gamintojas_gr, x='kaina', y='gamintojas', orient='h', hue='spalva')
# sns.boxplot(data=df_gamintojas, x='kaina', y='gamintojas', hue='spalva', showmeans=True, showfliers=False)
plt.title('Vidutinė šaldytuvų kaina pagal gamintoją (Varle.lt)')
plt.ylabel('')
plt.xlabel('Kaina')
plt.show()

In [ ]:
# Pigu
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT *
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

for col in df.columns:
    print(col)

In [ ]:
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,
`Montavimo tipas:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

df.head()

In [ ]:
c = df['Montavimo tipas:'].value_counts()
fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))
ax1.pie(c.values, labels=c.index, autopct='%.2f%%', startangle=90)
sns.boxplot(data=df, x='Montavimo tipas:', y='kaina', ax=ax2, showmeans=True, showfliers=False)
ax1.set_title('Šaldytuvai pagal montavimo tipą (Pigu.lt)')
ax2.set_title('Šaldytuvų kainos pasiskirtymas pagal montavimo tipą (Pigu.lt)')
fig.tight_layout()
plt.show()

In [ ]:
# Šaldytuvų tipai Pigu.lt
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,
`Šaldytuvo tipas:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

df.head()

In [ ]:
c = df['Šaldytuvo tipas:'].value_counts()
fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.pie(c.values, labels=c.index, autopct='%.2f%%', startangle=33, pctdistance=0.7, labeldistance = 0.9, textprops={'fontsize':8, 'color': 'black'})
sns.boxplot(data=df, x='Šaldytuvo tipas:', y='kaina', ax=ax2, showmeans=True, showfliers=False)
ax1.set_title('Šaldytuvai pagal  tipą (Pigu.lt)')
ax2.set_title('Šaldytuvų kainos pasiskirtymas pagal tipą (Pigu.lt)')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)

fig.tight_layout()
plt.show()

In [ ]:
# Šaldytuvų tūris Pigu.lt
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,
`Bendra talpa:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

df.head()

In [ ]:
def set_talpa(x):
    if x is not None:
        return float(x.replace('l', ''))
    
df['talpa'] = df['Bendra talpa:'].apply(set_talpa)
df.dropna(subset='talpa', inplace=True)
df['talpa2'] = df['talpa'].apply(lambda x: int(np.ceil(x/10) * 10))


In [ ]:
dfv = df[['kaina', 'talpa2']]

dfv_gr = dfv.groupby('talpa2').mean(numeric_only=True).round().reset_index()
dfv_gr.head()
sns.regplot(data=dfv_gr, x='talpa2', y='kaina', order=2)
plt.title('Šaldytuvo kainos priklausomybė nuo talpos (Pigu.lt)')
plt.ylabel('Kaina')
plt.xlabel('Šaldytuvo talpa (l)')
plt.show()

In [ ]:
# Šaldytuvų energijos klase Pigu.lt
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,
`Energijos klasė:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

df.head()

In [ ]:
df.dropna(subset='Energijos klasė:', inplace=True)
klases = sorted(list(set(df['Energijos klasė:'].tolist())))

sns.boxplot(data=df, x='Energijos klasė:', y='kaina', showmeans=True, showfliers=False, order=klases)
plt.title('Šaldytuvų klainos pasiskirtymas pagal energijos klasę (Pigu.lt)')
plt.xlabel('Energijos klasė')
plt.ylabel('Kaina')
plt.show()

In [ ]:
# Šaldytuvai energijos sanaudos Pigu.lt
SDB = sqlite3.connect('VarlePigu.db')
Cs = SDB.cursor()

sql="""SELECT kaina,
`Energijos sąnaudos per metus:`
FROM "SaldytuvaiPigu";
"""
df = pd.read_sql_query(sql, con=SDB)
SDB.close()

df.head()

In [ ]:
# def set_sanaudos(x):
#     if x is not None:
#         return float(x.replace('^[0-9]', ''))
    
# df['sanaudos'] = df['Energijos sąnaudos per metus:'].apply(set_sanaudos)

df['sanaudos'] = df['Energijos sąnaudos per metus:'].str.extract('(\d+)')
df.head()
# df.dropna(subset='talpa', inplace=True)
# df['talpa2'] = df['talpa'].apply(lambda x: int(np.ceil(x/10) * 10))

In [ ]:
df.dropna(subset='sanaudos', inplace=True)
df['sanaudos'] = df['sanaudos'].apply(lambda x: int(x))
c = df['sanaudos'].mean(numeric_only=True)
print(c)